![<caption>](UFPR.PNG)
    
    
IAA007 – Aprendizado de Máquina e Laboratório de IA    
**Prof. Dr. Jaime Wojciechowski**
    
Josué Leal Evangelista

### CLASSIFICAÇÃO

In [1]:
# DESATIVAR MENSAGENS DE WARNINGS (warn = -1) E ATIVAR (warn = 0)
options(warn = -1)
# NOTAÇÃO CIENTIFICA
options(scipen = 999)

#### CARREGANDO BIBLIOTECAS

In [2]:
suppressPackageStartupMessages({
library(caret)
library(e1071)
library(data.table)
library(dplyr)
library(ggplot2)
library(tidyr)
library(randomForest)
library(corrplot)
})

### 1 - CLASSIFICAÇÃO 

#### 1.1 - VEÍCULO

In [3]:
dados_veiculo <- fread("Material 02 - 5 - C - Veiculos - Dados.csv", stringsAsFactors = TRUE)
df_veiculo <- as.data.frame(dados_veiculo)

In [4]:
head(df_veiculo)

a,Comp,Circ,DCirc,RadRa,PrAxisRa,MaxLRa,ScatRa,Elong,PrAxisRect,MaxLRect,ScVarMaxis,ScVarmaxis,RaGyr,SkewMaxis,Skewmaxis,Kurtmaxis,KurtMaxis,HollRa,tipo
1,95,48,83,178,72,10,162,42,20,159,176,379,184,70,6,16,187,197,van
2,91,41,84,141,57,9,149,45,19,143,170,330,158,72,9,14,189,199,van
3,104,50,106,209,66,10,207,32,23,158,223,635,220,73,14,9,188,196,saab
4,93,41,82,159,63,9,144,46,19,143,160,309,127,63,6,10,199,207,van
5,85,44,70,205,103,52,149,45,19,144,241,325,188,127,9,11,180,183,bus
6,107,57,106,172,50,6,255,26,28,169,280,957,264,85,5,9,181,183,bus


In [5]:
df_veiculo$a <- NULL

In [6]:
# VERIFICANDO VALORES NA E NULL
any(is.na(df_veiculo))
any(is.null(df_veiculo))

[1] FALSE

[1] FALSE

In [7]:
# QUANTIDADE DE OBSERVAÇÕES DE CADA CLASSE
table(df_veiculo$tipo)


 bus opel saab  van 
 218  212  217  199 

In [8]:
set.seed(1971)

In [9]:
# DIVIDINDO DADOS TREINO E TESTE 80% e 20%
indices <- createDataPartition(df_veiculo$tipo, p = 0.8, list = FALSE)
dados_treino <- df_veiculo[indices,]
dados_teste <- df_veiculo[-indices,]

In [10]:
# NORMALIZAÇÃO PACOTE CARET
# method = c("center","scale") -> Padronização fórmula do score z
# method = c("range") -> Normalização fórmula do min-max

# TREINO
transforma1  <- preProcess(dados_treino[,c(1:18)], method = c("range"))
dados_treino_normalizado <- predict(transforma1, dados_treino)

# TESTE
transforma2  <- preProcess(dados_teste[,c(1:18)], method = c("range"))
dados_teste_normalizado <- predict(transforma2, dados_teste)

### 1.1 - REDES NEURAIS ARTIFICIAIS - RNA

#### APRENDIZADO - MODELO RNA - HOLD-OUT

In [11]:
modelo_rna <- train(tipo ~ MaxLRa + ScVarmaxis + MaxLRect + DCirc + ScatRa + Elong, 
                    dados_treino_normalizado, 
                    method = "nnet", 
                    trace = FALSE)
modelo_rna

Neural Network 

679 samples
  6 predictor
  4 classes: 'bus', 'opel', 'saab', 'van' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 679, 679, 679, 679, 679, 679, ... 
Resampling results across tuning parameters:

  size  decay   Accuracy   Kappa    
  1     0.0000  0.5088772  0.3500870
  1     0.0001  0.5219839  0.3658440
  1     0.1000  0.5148353  0.3566708
  3     0.0000  0.6739409  0.5653526
  3     0.0001  0.6623323  0.5502631
  3     0.1000  0.6694116  0.5595198
  5     0.0000  0.7038693  0.6044694
  5     0.0001  0.7022237  0.6024723
  5     0.1000  0.6988777  0.5979456

Accuracy was used to select the optimal model using the largest value.
The final values used for the model were size = 5 and decay = 0.

In [12]:
previsao_rna <- predict(modelo_rna, dados_teste_normalizado)

#### MÉTRICAS - MODELO RNA - HOLD-OUT

In [13]:
# MATRIZ DE CONFUSÃO
matriz_confusao1 <- confusionMatrix(previsao_rna, dados_teste_normalizado$tipo)
matriz_confusao1

Confusion Matrix and Statistics

          Reference
Prediction bus opel saab van
      bus   20    1    1   0
      opel   3    2    0   0
      saab   6   33   35   1
      van   14    6    7  38

Overall Statistics
                                               
               Accuracy : 0.5689               
                 95% CI : (0.4901, 0.6451)     
    No Information Rate : 0.2575               
    P-Value [Acc > NIR] : < 0.00000000000000022
                                               
                  Kappa : 0.4267               
                                               
 Mcnemar's Test P-Value : 0.00000000001706     

Statistics by Class:

                     Class: bus Class: opel Class: saab Class: van
Sensitivity              0.4651     0.04762      0.8140     0.9744
Specificity              0.9839     0.97600      0.6774     0.7891
Pos Pred Value           0.9091     0.40000      0.4667     0.5846
Neg Pred Value           0.8414     0.75309      0.9130    

#### APRENDIZADO - MODELO RNA - CROSS VALIDATION

In [14]:
crossvalidation1 <- trainControl(method = "cv", number = 10)
modelo_rna_cv <- train(tipo ~ MaxLRa + ScVarmaxis + MaxLRect + DCirc + ScatRa + Elong, 
                       dados_treino_normalizado, 
                       trainControl = crossvalidation1, 
                       method = "nnet", 
                       trace = FALSE)
modelo_rna_cv

Neural Network 

679 samples
  6 predictor
  4 classes: 'bus', 'opel', 'saab', 'van' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 679, 679, 679, 679, 679, 679, ... 
Resampling results across tuning parameters:

  size  decay   Accuracy   Kappa    
  1     0.0000  0.5241866  0.3714193
  1     0.0001  0.5270744  0.3746325
  1     0.1000  0.5213931  0.3667294
  3     0.0000  0.6712246  0.5624201
  3     0.0001  0.6645559  0.5534750
  3     0.1000  0.6659235  0.5558947
  5     0.0000  0.7018134  0.6024986
  5     0.0001  0.7011622  0.6015937
  5     0.1000  0.6895034  0.5864893

Accuracy was used to select the optimal model using the largest value.
The final values used for the model were size = 5 and decay = 0.

In [15]:
previsao_rna_cv <- predict(modelo_rna_cv, dados_teste_normalizado)

#### MÉTRICAS - MODELO RNA - CROSSVALIDATION

In [16]:
# MATRIZ DE CONFUSÃO
matriz_confusao2 <- confusionMatrix(previsao_rna_cv, dados_teste_normalizado$tipo)
matriz_confusao2

Confusion Matrix and Statistics

          Reference
Prediction bus opel saab van
      bus   19    1    1   0
      opel   8    4    2   0
      saab   2   31   34   1
      van   14    6    6  38

Overall Statistics
                                               
               Accuracy : 0.5689               
                 95% CI : (0.4901, 0.6451)     
    No Information Rate : 0.2575               
    P-Value [Acc > NIR] : < 0.00000000000000022
                                               
                  Kappa : 0.4268               
                                               
 Mcnemar's Test P-Value : 0.0000000005007      

Statistics by Class:

                     Class: bus Class: opel Class: saab Class: van
Sensitivity              0.4419     0.09524      0.7907     0.9744
Specificity              0.9839     0.92000      0.7258     0.7969
Pos Pred Value           0.9048     0.28571      0.5000     0.5938
Neg Pred Value           0.8356     0.75163      0.9091    

#### APRENDIZADO - MELHOR MODELO RNA - CROSS VALIDATION E TUNEGRID

In [17]:
tunegrid1 <- expand.grid(size = seq(from = 1, to = 10, by = 1), decay = seq(from = 0.1, to = 0.9, by = 0.1))
modelo_rna_best <- train(tipo ~ MaxLRa + ScVarmaxis + MaxLRect + DCirc + ScatRa + Elong, 
                         dados_treino_normalizado, 
                         method = "nnet", 
                         trainControl = crossvalidation1,
                         tuneGrid = tunegrid1, 
                         MaxNWts = 1000, 
                         maxit = 200, trace = FALSE)
modelo_rna_best

Neural Network 

679 samples
  6 predictor
  4 classes: 'bus', 'opel', 'saab', 'van' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 679, 679, 679, 679, 679, 679, ... 
Resampling results across tuning parameters:

  size  decay  Accuracy   Kappa    
   1    0.1    0.5222304  0.3672298
   1    0.2    0.5166855  0.3605124
   1    0.3    0.5141598  0.3577540
   1    0.4    0.5090883  0.3512908
   1    0.5    0.5006043  0.3403603
   1    0.6    0.4911609  0.3282421
   1    0.7    0.4798000  0.3134806
   1    0.8    0.4723115  0.3038105
   1    0.9    0.4642188  0.2932991
   2    0.1    0.6505188  0.5353036
   2    0.2    0.6393594  0.5207178
   2    0.3    0.6289368  0.5071758
   2    0.4    0.6312094  0.5101087
   2    0.5    0.6252076  0.5023442
   2    0.6    0.6203197  0.4960085
   2    0.7    0.6164944  0.4913569
   2    0.8    0.6115984  0.4850560
   2    0.9    0.6074247  0.4796875
   3    0.1    0.6679598  0.5580430
   3    0.2    0.6545878  0.54082

In [18]:
previsao_rna_best <- predict(modelo_rna_best, dados_teste_normalizado)

#### MÉTRICAS - MELHOR MODELO RNA - CROSS VALIDATION E TUNEGRID

In [19]:
# MATRIZ DE CONFUSÃO
matriz_confusao3 <- confusionMatrix(previsao_rna_best, dados_teste_normalizado$tipo)
matriz_confusao3

Confusion Matrix and Statistics

          Reference
Prediction bus opel saab van
      bus   32    1    2   0
      opel   1    9    3   1
      saab   8   28   32   1
      van    2    4    6  37

Overall Statistics
                                               
               Accuracy : 0.6587               
                 95% CI : (0.5814, 0.7302)     
    No Information Rate : 0.2575               
    P-Value [Acc > NIR] : < 0.00000000000000022
                                               
                  Kappa : 0.5449               
                                               
 Mcnemar's Test P-Value : 0.00002391           

Statistics by Class:

                     Class: bus Class: opel Class: saab Class: van
Sensitivity              0.7442     0.21429      0.7442     0.9487
Specificity              0.9758     0.96000      0.7016     0.9062
Pos Pred Value           0.9143     0.64286      0.4638     0.7551
Neg Pred Value           0.9167     0.78431      0.8878    

### 1.2 - K-NEAREST NEIGHBOR - KNN

#### APRENDIZADO - MODELO KNN - TUNEGRID

In [20]:
tunegrid2 <- expand.grid(k = c(3, 5, 7, 9))

In [21]:
modelo_knn <- train(tipo ~ MaxLRa + ScVarmaxis + MaxLRect + DCirc + ScatRa + Elong, 
                    dados_treino_normalizado, 
                    method = "knn", 
                    tuneGrid = tunegrid2)
modelo_knn

k-Nearest Neighbors 

679 samples
  6 predictor
  4 classes: 'bus', 'opel', 'saab', 'van' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 679, 679, 679, 679, 679, 679, ... 
Resampling results across tuning parameters:

  k  Accuracy   Kappa    
  3  0.6335245  0.5103983
  5  0.6443198  0.5252181
  7  0.6522833  0.5360853
  9  0.6568125  0.5423272

Accuracy was used to select the optimal model using the largest value.
The final value used for the model was k = 9.

In [22]:
previsao_knn <-predict(modelo_knn, dados_teste_normalizado)

#### MÉTRICAS - MODELO KNN - TUNEGRID

In [23]:
# MATRIZ DE CONFUSÃO
matriz_confusao4 <- confusionMatrix(previsao_knn, dados_teste_normalizado$tipo)
matriz_confusao4

Confusion Matrix and Statistics

          Reference
Prediction bus opel saab van
      bus   33    1    5   0
      opel   4   12   11   0
      saab   6   23   21   1
      van    0    6    6  38

Overall Statistics
                                               
               Accuracy : 0.6228               
                 95% CI : (0.5446, 0.6965)     
    No Information Rate : 0.2575               
    P-Value [Acc > NIR] : < 0.00000000000000022
                                               
                  Kappa : 0.4974               
                                               
 Mcnemar's Test P-Value : NA                   

Statistics by Class:

                     Class: bus Class: opel Class: saab Class: van
Sensitivity              0.7674     0.28571      0.4884     0.9744
Specificity              0.9516     0.88000      0.7581     0.9062
Pos Pred Value           0.8462     0.44444      0.4118     0.7600
Neg Pred Value           0.9219     0.78571      0.8103    

### 1.3 - SUPPORT VECTOR MACHINES - SVM

#### APRENDIZADO - MODELO SVM - HOLD-OUT

In [24]:
modelo_svm <- train(tipo ~ MaxLRa + ScVarmaxis + MaxLRect + DCirc + ScatRa + Elong, 
                    dados_treino_normalizado, 
                    method = "svmRadial")
modelo_svm

Support Vector Machines with Radial Basis Function Kernel 

679 samples
  6 predictor
  4 classes: 'bus', 'opel', 'saab', 'van' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 679, 679, 679, 679, 679, 679, ... 
Resampling results across tuning parameters:

  C     Accuracy   Kappa    
  0.25  0.6913006  0.5889020
  0.50  0.6961941  0.5951221
  1.00  0.6984187  0.5980035

Tuning parameter 'sigma' was held constant at a value of 0.6128511
Accuracy was used to select the optimal model using the largest value.
The final values used for the model were sigma = 0.6128511 and C = 1.

In [25]:
previsao_svm <- predict(modelo_svm, dados_teste_normalizado)

#### MÉTRICAS - MODELO SVM - HOLD-OUT

In [26]:
# MATRIZ DE CONFUSÃO
matriz_confusao5 <- confusionMatrix(previsao_svm, dados_teste_normalizado$tipo)
matriz_confusao5

Confusion Matrix and Statistics

          Reference
Prediction bus opel saab van
      bus   21    1    1   0
      opel   1    7    3   0
      saab  12   27   30   0
      van    9    7    9  39

Overall Statistics
                                               
               Accuracy : 0.5808               
                 95% CI : (0.5021, 0.6566)     
    No Information Rate : 0.2575               
    P-Value [Acc > NIR] : < 0.00000000000000022
                                               
                  Kappa : 0.4427               
                                               
 Mcnemar's Test P-Value : 0.0000000009271      

Statistics by Class:

                     Class: bus Class: opel Class: saab Class: van
Sensitivity              0.4884     0.16667      0.6977     1.0000
Specificity              0.9839     0.96800      0.6855     0.8047
Pos Pred Value           0.9130     0.63636      0.4348     0.6094
Neg Pred Value           0.8472     0.77564      0.8673    

#### APRENDIZADO - MODELO SVM - CROSS VALIDATION

In [27]:
crossvalidation2 <- trainControl(method = "cv", number = 15)
modelo_svm_cv <- train(tipo ~ MaxLRa + ScVarmaxis + MaxLRect + DCirc + ScatRa + Elong, 
                       dados_treino_normalizado, 
                       trainControl = crossvalidation2, 
                       method = "svmRadial")
modelo_svm_cv

Support Vector Machines with Radial Basis Function Kernel 

679 samples
  6 predictor
  4 classes: 'bus', 'opel', 'saab', 'van' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 679, 679, 679, 679, 679, 679, ... 
Resampling results across tuning parameters:

  C     Accuracy   Kappa    
  0.25  0.6971052  0.5970711
  0.50  0.7037224  0.6055395
  1.00  0.7075128  0.6104204

Tuning parameter 'sigma' was held constant at a value of 0.5762355
Accuracy was used to select the optimal model using the largest value.
The final values used for the model were sigma = 0.5762355 and C = 1.

In [28]:
previsao_svm_cv <- predict(modelo_svm_cv, dados_teste_normalizado)

#### MÉTRICAS - MODELO SVM - CROSS VALIDATION

In [29]:
# MATRIZ DE CONFUSÃO
matriz_confusao6 <- confusionMatrix(previsao_svm_cv, dados_teste_normalizado$tipo)
matriz_confusao6

Confusion Matrix and Statistics

          Reference
Prediction bus opel saab van
      bus   21    1    1   0
      opel   1    7    3   0
      saab  12   27   30   0
      van    9    7    9  39

Overall Statistics
                                               
               Accuracy : 0.5808               
                 95% CI : (0.5021, 0.6566)     
    No Information Rate : 0.2575               
    P-Value [Acc > NIR] : < 0.00000000000000022
                                               
                  Kappa : 0.4427               
                                               
 Mcnemar's Test P-Value : 0.0000000009271      

Statistics by Class:

                     Class: bus Class: opel Class: saab Class: van
Sensitivity              0.4884     0.16667      0.6977     1.0000
Specificity              0.9839     0.96800      0.6855     0.8047
Pos Pred Value           0.9130     0.63636      0.4348     0.6094
Neg Pred Value           0.8472     0.77564      0.8673    

#### APRENDIZADO - MODELO SVM - CROSS VALIDATION E TUNEGRID

In [30]:
tunegrid2 <- expand.grid(C = c(1, 2, 10, 50, 100), sigma = c(0.01, 0.015, 0.2))
modelo_svm_best <- train(tipo ~ MaxLRa + ScVarmaxis + MaxLRect + DCirc + ScatRa + Elong, 
                         dados_treino_normalizado, 
                         method = "svmRadial", 
                         trainControl = crossvalidation2,
                         tuneGrid = tunegrid2)
modelo_svm_best

Support Vector Machines with Radial Basis Function Kernel 

679 samples
  6 predictor
  4 classes: 'bus', 'opel', 'saab', 'van' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 679, 679, 679, 679, 679, 679, ... 
Resampling results across tuning parameters:

  C    sigma  Accuracy   Kappa    
    1  0.010  0.5911344  0.4585655
    1  0.015  0.6425115  0.5251039
    1  0.200  0.7085681  0.6114613
    2  0.010  0.6480809  0.5323986
    2  0.015  0.6642288  0.5536408
    2  0.200  0.7122105  0.6163207
   10  0.010  0.6841530  0.5791722
   10  0.015  0.6926913  0.5903829
   10  0.200  0.7196160  0.6260056
   50  0.010  0.6994752  0.5991911
   50  0.015  0.7031318  0.6040271
   50  0.200  0.7334384  0.6441448
  100  0.010  0.7035860  0.6045805
  100  0.015  0.7078561  0.6103929
  100  0.200  0.7345647  0.6456506

Accuracy was used to select the optimal model using the largest value.
The final values used for the model were sigma = 0.2 and C = 100.

In [31]:
previsao_svm_best <- predict(modelo_svm_best, dados_teste_normalizado)

#### MÉTRICAS - MODELO SVM - CROSS VALIDATION E TUNEGRID

In [32]:
# MATRIZ DE CONFUSÃO
matriz_confusao7 <- confusionMatrix(previsao_svm_best, dados_teste_normalizado$tipo)
matriz_confusao7 

Confusion Matrix and Statistics

          Reference
Prediction bus opel saab van
      bus   23    1    1   0
      opel   8   17   19   0
      saab   7   17   18   1
      van    5    7    5  38

Overall Statistics
                                               
               Accuracy : 0.5749               
                 95% CI : (0.4961, 0.6509)     
    No Information Rate : 0.2575               
    P-Value [Acc > NIR] : < 0.00000000000000022
                                               
                  Kappa : 0.4346               
                                               
 Mcnemar's Test P-Value : 0.0003843            

Statistics by Class:

                     Class: bus Class: opel Class: saab Class: van
Sensitivity              0.5349      0.4048      0.4186     0.9744
Specificity              0.9839      0.7840      0.7984     0.8672
Pos Pred Value           0.9200      0.3864      0.4186     0.6909
Neg Pred Value           0.8592      0.7967      0.7984    

### 1.4 - RANDOM FOREST

#### APRENDIZADO - MODELO RANDOM FOREST - HOLD-OUT

In [33]:
modelo_rf <- train(tipo ~ MaxLRa + ScVarmaxis + MaxLRect + DCirc + ScatRa + Elong, dados_treino, method = "rf")
modelo_rf

Random Forest 

679 samples
  6 predictor
  4 classes: 'bus', 'opel', 'saab', 'van' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 679, 679, 679, 679, 679, 679, ... 
Resampling results across tuning parameters:

  mtry  Accuracy   Kappa    
  2     0.6921232  0.5892092
  4     0.6870558  0.5824491
  6     0.6838945  0.5781749

Accuracy was used to select the optimal model using the largest value.
The final value used for the model was mtry = 2.

In [34]:
previsao_rf <- predict(modelo_rf, dados_teste)

#### MÉTRICAS - MODELO RANDOM FOREST - HOLD-OUT

In [35]:
# MATRIZ DE CONFUSÃO
matriz_confusao8 <- confusionMatrix(previsao_rf, dados_teste$tipo)
matriz_confusao8

Confusion Matrix and Statistics

          Reference
Prediction bus opel saab van
      bus   41    3    4   0
      opel   1   19   15   0
      saab   1   15   22   1
      van    0    5    2  38

Overall Statistics
                                               
               Accuracy : 0.7186               
                 95% CI : (0.6439, 0.7853)     
    No Information Rate : 0.2575               
    P-Value [Acc > NIR] : < 0.00000000000000022
                                               
                  Kappa : 0.6249               
                                               
 Mcnemar's Test P-Value : NA                   

Statistics by Class:

                     Class: bus Class: opel Class: saab Class: van
Sensitivity              0.9535      0.4524      0.5116     0.9744
Specificity              0.9435      0.8720      0.8629     0.9453
Pos Pred Value           0.8542      0.5429      0.5641     0.8444
Neg Pred Value           0.9832      0.8258      0.8359    

#### APRENDIZADO - MODELO RANDOM FOREST - CROSS VALIDATION

In [36]:
crossvalidation3 <- trainControl(method = "cv", number = 8)
modelo_rf_cv <- train(tipo ~ MaxLRa + ScVarmaxis + MaxLRect + DCirc + ScatRa + Elong, 
                      dados_treino, 
                      method = "rf",
                      trainControl = crossvalidation3)
modelo_rf_cv

Random Forest 

679 samples
  6 predictor
  4 classes: 'bus', 'opel', 'saab', 'van' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 679, 679, 679, 679, 679, 679, ... 
Resampling results across tuning parameters:

  mtry  Accuracy   Kappa    
  2     0.6986922  0.5979512
  4     0.6944076  0.5921705
  6     0.6910297  0.5877064

Accuracy was used to select the optimal model using the largest value.
The final value used for the model was mtry = 2.

In [37]:
previsao_rf_cv <- predict(modelo_rf_cv, dados_teste)

#### MÉTRICAS - MODELO RANDOM FOREST - CROSS VALIDATION

In [38]:
# MATRIZ DE CONFUSÃO
matriz_confusao9 <- confusionMatrix(previsao_rf_cv, dados_teste$tipo)
matriz_confusao9

Confusion Matrix and Statistics

          Reference
Prediction bus opel saab van
      bus   41    3    4   0
      opel   1   18   15   0
      saab   1   16   22   1
      van    0    5    2  38

Overall Statistics
                                               
               Accuracy : 0.7126               
                 95% CI : (0.6376, 0.7799)     
    No Information Rate : 0.2575               
    P-Value [Acc > NIR] : < 0.00000000000000022
                                               
                  Kappa : 0.6169               
                                               
 Mcnemar's Test P-Value : NA                   

Statistics by Class:

                     Class: bus Class: opel Class: saab Class: van
Sensitivity              0.9535      0.4286      0.5116     0.9744
Specificity              0.9435      0.8720      0.8548     0.9453
Pos Pred Value           0.8542      0.5294      0.5500     0.8444
Neg Pred Value           0.9832      0.8195      0.8346    

#### APRENDIZADO - MODELO RANDOM FOREST - CROSS VALIDATION E TUNEGRID

In [39]:
tunegrid3 <- expand.grid(mtry = c(2, 4, 7, 9))
modelo_rf_best <- train(tipo ~ MaxLRa + ScVarmaxis + MaxLRect + DCirc + ScatRa + Elong, 
                        dados_treino, 
                        method = "rf", 
                        trainControl = crossvalidation3, 
                        tuneGrid = tunegrid3)
modelo_rf_best

Random Forest 

679 samples
  6 predictor
  4 classes: 'bus', 'opel', 'saab', 'van' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 679, 679, 679, 679, 679, 679, ... 
Resampling results across tuning parameters:

  mtry  Accuracy   Kappa    
  2     0.6964600  0.5950245
  4     0.6890600  0.5851823
  7     0.6859066  0.5810024
  9     0.6881607  0.5840013

Accuracy was used to select the optimal model using the largest value.
The final value used for the model was mtry = 2.

In [40]:
previsao_rf_best <- predict(modelo_rf_best, dados_teste)

#### MÉTRICAS - MODELO RANDOM FOREST - CROSS VALIDATION E TUNEGRID

In [41]:
# MATRIZ DE CONFUSÃO
matriz_confusao10 <- confusionMatrix(previsao_rf_best, dados_teste$tipo)
matriz_confusao10

Confusion Matrix and Statistics

          Reference
Prediction bus opel saab van
      bus   41    3    5   0
      opel   1   19   15   0
      saab   1   15   21   1
      van    0    5    2  38

Overall Statistics
                                               
               Accuracy : 0.7126               
                 95% CI : (0.6376, 0.7799)     
    No Information Rate : 0.2575               
    P-Value [Acc > NIR] : < 0.00000000000000022
                                               
                  Kappa : 0.6169               
                                               
 Mcnemar's Test P-Value : NA                   

Statistics by Class:

                     Class: bus Class: opel Class: saab Class: van
Sensitivity              0.9535      0.4524      0.4884     0.9744
Specificity              0.9355      0.8720      0.8629     0.9453
Pos Pred Value           0.8367      0.5429      0.5526     0.8444
Neg Pred Value           0.9831      0.8258      0.8295    